In [ ]:
from datetime import datetime
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

In [ ]:
hoy = datetime.now().strftime("%d.%m.%Y")
ayer = datetime.now() - pd.DateOffset(days=1)
ayer = ayer.strftime("%d.%m.%Y")

print("Hoy: ", hoy)
print("Ayer: ", ayer)

In [ ]:
base_pagos_path = "Base Pagos " + ayer + ".xlsx"  
asignacion_path = "base_asignacion_20240809.xlsx"
monoproducto = "monoproducto/MONOPRODUCTO " + hoy + ".xlsx"
multiproducto = "multiproducto/MULTIPRODUCTO " + hoy + ".xlsx"
reactiva = "reactiva/REACTIVA " + hoy + ".xlsx"

In [ ]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

In [ ]:
df_base['FECHA_ENVIO'] = pd.to_datetime('today').strftime('%d-%b')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype("Int64").astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base["CLAVSERV"].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

In [ ]:
df_base.isnull().sum()

In [ ]:
#################### ASIGNACION ####################

In [ ]:
df_asignacion = pd.read_excel(asignacion_path)
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

In [ ]:
#################### COPIAS ####################

In [ ]:
df_base_test = df_base.copy()
print("Base", df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print("Asignacion", df_asignacion_test.shape)

In [ ]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype("Int64").astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})

df_base_test["TIPO_CARTERA"] = df_base_test["TIPO_CARTERA"].fillna("NULL")
df_base_test["TIPO_FONDO"] = df_base_test["TIPO_FONDO"].fillna("NULL")

df_base_test["FLAG"] = df_base_test["FLAG"].astype("Int64")
df_base_test["CONTRATO"] = df_base_test["CONTRATO"].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

In [ ]:
#################### MONOPRODUCTO ####################

In [ ]:
df_mono = df_base_test[df_base_test["FLAG"] == 1]
print(df_mono.shape)
df_mono.head(5)

In [ ]:
df_mono["TIPO_CARTERA"].value_counts()

In [ ]:
df_mono["TIPO_FONDO"].value_counts()

In [ ]:
df_mono[["TIPO_CARTERA", "TIPO_FONDO"]].value_counts()

In [ ]:
#################### MONO REACTIVA ####################

In [ ]:
df_reactiva = df_mono[(df_mono["TIPO_CARTERA"] == "UNSECURED") & (df_mono["TIPO_FONDO"] == "REACTIVA")]
print(df_reactiva.shape)
df_reactiva.head(5)

In [ ]:
df_reactiva.to_excel(reactiva, index=False)

In [ ]:
#################### FIN REACTIVA ####################

In [ ]:
df_mono = df_mono[df_mono["TIPO_CARTERA"] == "UNSECURED"]
df_mono = df_mono[df_mono["TIPO_FONDO"] == "NULL"]
print(df_mono.shape)

In [ ]:
df_mono["CONTRATO"] = df_mono["CONTRATO"].astype(str).str.zfill(18)

In [ ]:
df_mono.to_excel(monoproducto, index=False)

In [ ]:
#################### FIN MONOPRODUCTO ####################

In [ ]:
#################### MULTIPRODUCTO ####################

In [ ]:
df_multi = df_base_test[df_base_test["FLAG"] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

In [ ]:
df_multi = df_multi[df_multi["TIPO_CARTERA"] == "UNSECURED"]
df_multi = df_multi[df_multi["MONEDA"] == "PEN"]

df_multi["CONTRATO"] = None
df_multi["TIPO_FONDO"] = None
df_multi["CARTERA"] = None
df_multi = df_multi.drop(columns=['TIPO_CARTERA', 'AGENCIA'])
df_multi = df_multi.drop_duplicates()

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

print(df_multi.shape)
df_multi.head(5)

In [ ]:
df_multi.to_excel(multiproducto, index=False)

In [ ]:
#################### FIN MULTIPRODUCTO ####################

In [ ]:
#################### ABRIR ARCHIVOS ####################

In [ ]:
os.startfile(os.path.abspath(monoproducto))
os.startfile(os.path.abspath(multiproducto))
os.startfile(os.path.abspath(reactiva))

In [ ]:
#################### FIN ####################